# Anthropic function calling with Langchain Agents

In this notebook we leveraage Langchain agents to perform function calling tasks. 

First we install the necessary python packages such as `duckduckgo-search`, `wikipedia`, and `xmltodict`, amongst others. 

- Initialize connection to **anthropic.claude-v2** using Bedrock
- Define and test the following tools individually
  - Wikipedia
  - Web Search and
  - PubMed search
- Finally, we put this all together by initializing a Langchain agent executor

In [5]:
!pip install -U duckduckgo-search wikipedia unstructured xmltodict --quiet

In [6]:
import json
import warnings

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT, AnthropicBedrock
from IPython.display import Markdown
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun, PubmedQueryRun, tool
from langchain.utilities import WikipediaAPIWrapper
from rich import print
from utils import get_inference_parameters

%load_ext rich
warnings.filterwarnings("ignore")

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [7]:
region = "us-west-2"
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime", region_name=region
)  # bedrock runtime boto3 client
bedrock_model_id = "anthropic.claude-v2"  # Amazon Bedrock Model ID for claude-v2

# anthropic claude model parameters
model_kwargs = get_inference_parameters("anthropic")

# print(model_kwargs)

# Define the llm with langchain.llms.bedrock
bedrock_llm = Bedrock(
    model_id=bedrock_model_id, client=bedrock_runtime, model_kwargs=model_kwargs
)

## Let's define a few tools

- [Wikipedia](https://wikipedia.org/) (Search wikipedia pages)
- Web Search (Search web using [DuckDuckGo](https://duckduckgo.com/) search engine)
- [PubMed](https://pubmed.ncbi.nlm.nih.gov/) (Medical papers and articles)

#### 01.Wikipedia Tool

[Wikipedia](https://wikipedia.org/) is a multilingual free online encyclopedia written and maintained by a community of volunteers, through open collaboration. Wikipedia is the largest and most-read reference work in history. 

>*NOTE:* To use this tool, install `wikipedia` package.

In [8]:
# Define the API wrapper for Wikipedia
wikipedia = WikipediaAPIWrapper()

# Define the Wikipedia tool with a description and the function to retrieve results from Wikipedia
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
)

# Test the defined Wikipedia tool
# Markdown(wikipedia_tool("What is the Archimedes Principle?"))

---

#### 02.DuckDuckGo Search Tool

[DuckDuckGo](https://duckduckgo.com/) is an internet privacy company most popularly known for their private search engine. The company emphasizes privacy and anonimity as one of the key principles behind all their products.

>*NOTE:* To use this tool, install `duckduckgo-search` package.

In [9]:
# Define the API wrapper for DuckDuckGo search
duckduckgo_search = DuckDuckGoSearchRun()

# Define the DuckDuckGo tool using a description and the function to retrieve results from DuckDuckGo search
duckduckgo_tool = Tool(
    name="DuckDuckGoSearch",
    func=duckduckgo_search.run,
    description="useful for when you need to answer questions about current events",
)

# Test the DuckDuckGo tool
# Markdown(duckduckgo_tool("Weather in Seattle, Washington?"))

---

#### 03.PubMed Tool

[PubMed](https://pubmed.ncbi.nlm.nih.gov/) is a free search engine for medical papers and articles.

>*NOTE:* To use this tool, install `xmltodict` package.

In [10]:
# Define the API wrapper for PubMed search
pubmed_search = PubmedQueryRun()

# Define the PubMed tool using a description and the function to retrieve results from PubMed
pubmed_tool = Tool(
    name="PubmedQueryRun",
    func=pubmed_search.run,
    description="Useful for when you need medical information",
)

# Test PubMed tool
# Markdown(pubmed_tool("Covid diagnosis"))

## Putting it all together

- Define all the tools as a list
- Initialize the agent with tool list and `AgentType` of type `ZERO_SHOT_REACT_DESCRIPTION`
- Explicity tell the agent to handle parsing errors

In [11]:
# Define a list of tools for the agent
tools = [wikipedia_tool, duckduckgo_tool, pubmed_tool]

# Initialize the agent with access to all the tooks in the list
agent_executor = initialize_agent(
    tools,
    bedrock_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

output_dict = agent_executor.invoke(
    {"input": "Tell me about SageMaker serverless inference endpoints"}
)
print(output_dict)



> Entering new AgentExecutor chain...
 Here are the key points about SageMaker serverless inference endpoints:

Action: wikipedia
Action Input: Amazon SageMaker
Observation: Page: Amazon SageMaker
Summary: Amazon SageMaker is a cloud based machine-learning platform that allows the creation, training, and deployment by developers of machine-learning (ML) models on the cloud. It can be used to deploy ML models on embedded systems and edge-devices. SageMaker was launched in November 2017.

Page: Amazon Polly
Summary: Amazon Polly is a cloud service by Amazon Web Services, a subsidiary of Amazon.com, that converts text into spoken audio. It allows developers to create speech-enabled applications and products. It was launched in November 2016 and now includes 60 voices across 29 languages, some of which are Neural Text-to-Speech voices of higher quality. Users include Duolingo, a language education platform.

Page: GPT-J
Summary: GPT-J or GPT-J-6B is an open-source large language model (L

{
    'input': 'Tell me about SageMaker serverless inference endpoints',
    'output': 'SageMaker serverless inference endpoints allow deploying ML models without managing servers, 
auto-scale based on traffic, are cost-effective with pay per use pricing, can be invoked via HTTPS requests, are 
best for intermittent workloads needing low latency, support major frameworks, but have some limitations like max 
container size and request time.'
}